In [2]:
from keras import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

Using TensorFlow backend.


In [ ]:

train_csv = pd.read_csv("Data/train.csv")
train_csv_2 = train_csv.drop(columns=['target'])
print(train_csv_2.shape)
train_csv_2 = train_csv_2.append(pd.read_csv("Data/test_independent.csv"))
train_csv_one_hot = pd.get_dummies(train_csv_2)

train_csv_one_hot

scaled_data = MinMaxScaler().fit_transform(train_csv_one_hot)
print(scaled_data.shape)

model = Sequential()

enc = Sequential()
enc.add(Dense(300, activation='relu'))
enc.add(Dense(150, activation='relu'))
enc.add(Dense(10, name='embeddings', activation='relu'))

dec = Sequential()
dec.add(Dense(100, activation='relu'))
dec.add(Dense(150, activation='relu'))
dec.add(Dense(102, name='output'))


model.add(enc)
model.add(dec)

model.compile(optimizer='adam', loss='mse')
model.fit(scaled_data, scaled_data, epochs=5)

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

encodings = enc.predict(scaled_data)

pca = PCA()
principalComponents = pca.fit_transform(encodings)
print(pca.explained_variance_ratio_)
x_val = []
y_val= []
for i in range(principalComponents.shape[0]):
    x_val.append(principalComponents[i][0])
    y_val.append(principalComponents[i][1])
x_val = np.array(x_val)
y_val = np.array(y_val)

plt.scatter(x=x_val, y=y_val, alpha=0.5)

plt.legend(labels=np.arange(0,9))
plt.show()
from sklearn.cluster import AgglomerativeClustering

n_clusters = 2
cluster = AgglomerativeClustering(n_clusters=n_clusters, 
                                  affinity='euclidean',
                                  linkage='ward')
cl_nom = cluster.fit_predict(principalComponents)
targets = pd.read_csv("Data/train.csv")['target']
print(np.sum(abs(cl_nom[:13741] - targets )))
pred = pd.read_csv("Data/Pred.csv")['target']

print(np.sum(abs(cl_nom[13741:] - pred)))


In [3]:
train_csv = pd.read_csv('Data/train.csv')
train_csv['day']=[d.split('-')[0] for d in train_csv.date]
train_csv['month']=[d.split('-')[1] for d in train_csv.date]
train_csv['year']=[d.split('-')[2] for d in train_csv.date]
train=train_csv.drop(['date'], axis=1)

wind_in_dummies = pd.get_dummies(train['wind'])
month_in_dummies = pd.get_dummies(train['month'])

train_with_dummies = pd.concat([train, wind_in_dummies, month_in_dummies], axis=1)

train_with_dummies=train_with_dummies.drop(['wind', 'month'], axis=1)

X = train.iloc[:,1:]
int_columns = X.select_dtypes(exclude=['object']).columns

mean = train_with_dummies[int_columns].mean()
sd = train_with_dummies[int_columns].std()

x = 3
train_with_dummies[int_columns] = train_with_dummies[int_columns][ np.abs(train_with_dummies[int_columns] - mean) <= x * sd ]
mean = train_with_dummies[int_columns].mean()

train_with_dummies.replace(np.nan, mean, inplace=True)

print(train_with_dummies['carb_monox'])

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

train_scaled = StandardScaler().fit_transform(train_with_dummies)
train_scaled_df = pd.DataFrame(train_scaled, columns=train_with_dummies.columns) 
#y_scaled = StandardScaler().fit_transform(y)
X = train_scaled_df.copy() #if I want to use all variables
X = X.drop(columns=['carb_monox'])
y = train_scaled_df['carb_monox']
# print(train_scaled_df.shape)
# print(X.shape)
# print(y)

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.25,
                                                    random_state=123)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


0         200.000
1        2100.000
2         700.000
3        1222.923
4        1300.000
           ...   
13995     400.000
13996    1600.000
13997    1700.000
13998    1000.000
13999     400.000
Name: carb_monox, Length: 14000, dtype: float64


(10500, 40) (10500,) (3500, 40) (3500,)


In [40]:
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

regressor = LinearRegression() #(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)[source]¶
parameters = {}
#print(sk.metrics.SCORERS.keys())
#DEFINE YOUR GRIDSEARCH 
gs = GridSearchCV(regressor, parameters,cv=3, scoring='neg_mean_absolute_error') #with no params it reduces to a CV
gs = gs.fit(X_train,y_train)

print('***GRIDSEARCH RESULTS***')
print("Best score: %f using %s" % (gs.best_score_, gs.best_params_))
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
params = gs.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


#gs.score(X_train, y_train)
gs.score(X_test, y_test)

print("MAE train: ", metrics.mean_absolute_error(y_train, gs.predict(X_train))) 
print("r2: ",metrics.r2_score(y_train, gs.predict(X_train)))

print("MAE test: ", metrics.mean_absolute_error(y_test, gs.predict(X_test))) 
print("r2: ",metrics.r2_score(y_test, gs.predict(X_test)))
#i check both test and train errors to check i am not doing overfitting

***GRIDSEARCH RESULTS***
Best score: -0.400159 using {}
-0.400159 (0.008017) with: {}
MAE train:  0.3976412039303451
r2:  0.6291918369584113
MAE test:  0.40302826426030386
r2:  0.6404389897371718


In [23]:

enc = Sequential()

# The Input Layer :
enc.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
enc.add(Dense(256, kernel_initializer='normal',activation='relu'))
enc.add(Dense(256, kernel_initializer='normal',activation='relu'))
enc.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
enc.add(Dense(1, kernel_initializer='normal',activation='linear'))



NN_model = Sequential()
NN_model.add(enc)
# Compile the network :
NN_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
NN_model.summary()

NN_model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_22 (Sequential)   (None, 1)                 170113    
Total params: 170,113
Trainable params: 170,113
Non-trainable params: 0
_________________________________________________________________


Train on 10500 samples, validate on 3500 samples
Epoch 1/5


  128/10500 [..............................] - ETA: 1:53 - loss: 1.1679 - acc: 0.0000e+00

 1152/10500 [==>...........................] - ETA: 11s - loss: 0.8453 - acc: 0.0000e+00 

 1792/10500 [====>.........................] - ETA: 7s - loss: 0.7092 - acc: 0.0000e+00 

...

 8960/10500 [========================>.....] - ETA: 0s - loss: 0.2775 - acc: 0.0000e+00

 9600/10500 [==========================>...] - ETA: 0s - loss: 0.2787 - acc: 0.0000e+00

10240/10500 [============================>.] - ETA: 0s - loss: 0.2779 - acc: 0.0000e+00

10496/10500 [============================>.] - ETA: 0s - loss: 0.2770 - acc: 0.0000e+00

10500/10500 [==============================] - 2s 154us/step - loss: 0.2770 - acc: 0.0000e+00 - val_loss: 0.3097 - val_acc: 0.0000e+00


In [17]:
encs = enc.predict(X_test)
print(encs.shape)

(3500, 10)


In [18]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def plot_pca(title, type): 
    x_val = []
    y_val= []
    for i in range(principalComponents.shape[0]):
        x_val.append(principalComponents[i][0])
        y_val.append(principalComponents[i][1])
    x_val = np.array(x_val)
    y_val = np.array(y_val)
    
    plt.scatter(x=x_val, y=y_val, alpha=0.5)
    plt.title(str(title)+""+type)
    plt.show()
    
titles=['z_mean','z_log_var','z']
scaler = StandardScaler()
enc_input = scaler.fit_transform(encs) 
pca = PCA(2)
principalComponents = pca.fit_transform(enc_input)
print(pca.explained_variance_ratio_)
plot_pca('CLUSTERS', '')

[0.37750328 0.20336105]
